In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在GCP上的端到端机器学习：MLOps阶段2：实验：开始使用AutoML训练

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_automl_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_automl_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_automl_training.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在Google Cloud上使用Vertex AI进行端到端MLOps生产。本教程涵盖了第2阶段：实验：开始使用AutoML训练。

### 目标

在本教程中，您将学习如何使用 `AutoML` 在 `Vertex AI` 上进行训练。

该教程使用以下谷歌云ML服务：

- `AutoML 训练`
- `Vertex AI 数据集`

执行的步骤包括：

- 训练图像模型
- 将图像模型导出为边缘模型
- 训练表格模型
- 将表格模型导出为云模型
- 训练文本模型
- 训练视频模型

### 推荐

在 Google Cloud 上进行端到端 MLOps 时，在何时使用 AutoML 时，以下是最佳实践：

* **您只有有限数量的训练数据**

* **在尝试自定义模型之前，您想建立一个基准度量**

### 数据集

#### 图像

本教程中使用的图像数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)的[花卉数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。本教程中的数据集版本存储在公共云存储桶中。训练的模型可以预测给定图像中的花卉类型，包括五种花卉：雏菊、蒲公英、玫瑰、向日葵或郁金香。

#### 表格

本教程中使用的表格数据集是来自[Google BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)的GSOD数据集。在本教程中使用的数据集版本仅使用年份、月份和日期这些字段来预测平均每日温度（mean_temp）的值。

#### 文本

本教程中使用的文本数据集是来自[Kaggle数据集](https://www.kaggle.com/ritresearch/happydb)的[Happy Moments数据集](https://www.kaggle.com/ritresearch/happydb)。在本教程中使用的数据集版本存储在公共云存储桶中。

#### 视频

本教程中使用的视频数据集是来自[MIT](http://cbcl.mit.edu/publications/ps/Kuehne_etal_iccv11.pdf)的[人体运动数据集](https://todo)中的高尔夫挥杆识别部分。在本教程中使用的数据集版本存储在公共云存储桶中。训练的模型可以预测高尔夫挥杆开始的起始帧。

### 费用
本教程使用 Google Cloud 的可计费组件：

- Vertex AI
- 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 基于您的预期使用量生成成本估算。

安装执行此笔记本所需的软件包。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install the packages

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-storage $USER_FLAG -q

重新启动核心

安装了额外的包之后，您需要重新启动笔记本核心，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的 Google Cloud 项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个账户时，您将获得 $300 的免费信用额度，用于支付计算/存储成本。

1. [确保您的项目已启用计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI、Compute Engine 和 Cloud Storage APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage_component)。

1. 如果您是在本地运行这个笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并且它会将以 `$` 为前缀的 Python 变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用 `gcloud` 获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于笔记本的其余部分操作。以下是支持Vertex AI的区域。我们建议您选择最靠近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不能使用多区域存储桶进行Vertex AI的训练。并非所有区域都支持所有的Vertex AI服务。

了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在进行实时教程会话，您可能正在使用共享测试帐户或项目。为避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在此教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google Cloud 帐号

**如果您正在使用 Vertex AI Workbench 笔记本**，则您的环境已经通过验证。跳过此步骤。

**如果您正在使用 Colab**，运行下面的单元格，并按照提示进行身份验证，通过 oAuth 认证您的帐号。

**否则**，请按照以下步骤操作：

在 Cloud 控制台中，转到 [创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

1. **点击创建服务帐号**。

2. 在 **服务帐号名称** 栏中输入一个名称，然后点击 **创建**。

3. 在 **将此服务帐号授权给项目** 部分，点击角色下拉列表。在过滤框中输入 "Vertex AI"，选择 **Vertex AI 管理员**。在过滤框中输入 "Storage Object Admin"，选择 **存储对象管理员**。

4. 点击创建。一个包含您的密钥的 JSON 文件将下载到您的本地环境中。

5. 将您的服务帐号密钥路径作为 GOOGLE_APPLICATION_CREDENTIALS 变量输入到下面的单元格中，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论你使用什么笔记本环境，以下步骤都是必须的。**

在初始化用于 Python 的 Vertex SDK 时，您需要指定一个云存储临时桶。这个临时桶是您数据集和模型资源相关数据在不同会话之间保留的地方。

在下面设置您的云存储桶的名称。云存储桶的命名必须在所有 Google Cloud 项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的存储桶不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查内容来验证对您的云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在整个教程中使用的变量。
### 导入库并定义常量

In [1]:
import google.cloud.aiplatform as aiplatform

### 初始化 Vertex AI SDK for Python

为您的项目和相应的存储桶初始化 Vertex AI SDK for Python。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## 自动机器学习训练任务

自动机器学习可以用来自动训练各种图像模型类型。自动机器学习自动化以下过程：

- 数据集预处理
- 特征工程
- 数据喂养
- 模型架构选择
- 超参数调整
- 训练模型

了解更多关于[Vertex AI for AutoML 用户](https://cloud.google.com/vertex-ai/docs/start/automl-users) 的信息。

自动机器学习图像模型

AutoML可以训练以下类型的图像模型：

- 分类
- 目标检测
- 分割

模型可以训练用于部署到云端，也可以导出到边缘设备。

了解更多关于[AutoML模型类型](https://cloud.google.com/vertex-ai/docs/start/automl-model-types)。

数据准备

用于图像的Vertex `Dataset`资源对您的数据有一些要求：

- 图像必须存储在Cloud Storage存储桶中。
- 每个图像文件必须是图像格式（PNG，JPEG，BMP，...）。
- 必须在您的Cloud Storage存储桶中存储一个包含每个图像的路径和标签的索引文件。
- 索引文件必须是CSV或JSONL格式。

了解更多关于[准备图像数据](https://cloud.google.com/vertex-ai/docs/datasets/prepare-image)

CSV

对于图像分类，CSV索引文件具有以下要求：

- 没有标题。
- 第一列是图像的云存储路径。
- 第二列是标签。
- 任何剩余的列都是用于多标签图像分类的附加标签。

对于图像目标检测，CSV索引文件具有以下要求：

- 没有标题。
- 第一列是图像的云存储路径。
- 第二列是标签。
- 第三/第四列是边界框的左上角。坐标已归一化，介于0和1之间。
- 第五/第六/第七列未使用，应为0。
- 第八/第九列是边界框的右下角。

ML_USE

在数据集用于训练时，每行可能额外指定将数据项指定为哪种拆分；否则，数据集将被随机拆分：80/10/10。

`ml_use` 分配由列的指定分配指定 -- 作为第一列。该值可以是以下之一：training、test 或 validation。

JSONL

对于图像分类，JSONL索引文件具有以下要求：

- 每个数据项都是单独的JSON对象，放在单独的一行上。
- 键/值对`image_gcs_uri`是图像的云存储路径。
- 键/值对`display_name`是图像的标签。

    { 'image_gcs_uri': image, 
      'classification_annotations': 
          { 'display_name': label
          }
    }
    
对于多标签，标签被指定为`display_name`键/值对的列表：

    { 'image_gcs_uri': image, 
      'classification_annotations': [
          { 'display_name': label1
          },
          { 'display_name': labelN
          },
       ]
    }
    
对于目标检测，JSONL索引文件具有以下要求：

- 每个数据项都是单独的JSON对象，放在单独的一行上。
- 键/值对`image_gcs_uri`是图像的云存储路径。
- 键/值对`bounding_box_annotations`是一个列表：
    - `display_name`：物体的标签
    - `x_min`、`y_min`、`x_max`、`y_max`：边界框的坐标

{
  "image_gcs_uri": image,
  "bounding_box_annotations": [
    {
      "display_name": label,
      "x_min": "X_MIN",
      "y_min": "Y_MIN",
      "x_max": "X_MAX",
      "y_max": "Y_MAX"
      }
    },
    {
      "display_name": "OBJECT2_LABEL",
      "x_min": "X_MIN",
      "y_min": "Y_MIN",
      "x_max": "X_MAX",
      "y_max": "Y_MAX"
    }
  ]
}


对于图像分割，JSONL索引文件具有以下要求：

- 每个数据项都是单独的JSON对象，放在单独的一行上。
- 键/值对`image_gcs_uri`是图像的云存储路径。
- 键/值对`category_mask_uri`是mask图像的云存储路径，格式为PNG。
- 键/值对`annotation_spec_colors`是一个列表，将mask颜色映射到一个标签。
  - 键/值对`display_name`是像素颜色mask的标签。
  - 键/值对`color`是对应标签的mask的RGB归一化像素值（在0和1之间）。

    { 'image_gcs_uri': image, 
      'segmentation_annotations': { 'category_mask_uri': mask_image, 'annotation_spec_colors' : [ 
          { 'display_name': label, 'color': {"red": value, "blue": value, "green": value} }, ...
      ] 
    }
    
ML_USE

每个JSONL对象还可以指定在拆分数据集进行训练时将数据项分配给哪个拆分；否则，随机拆分数据集: 80/10/10。

"data_item_resource_labels": {
      "aiplatform.googleapis.com/ml_use": "training|test|validation"
    }

*注意*：字典键字段也可以使用驼峰命名。例如，'image_gcs_uri'也可以是'imageGcsUri'。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

快速查看您的数据

本教程使用存储在公共云存储桶中的Happy Moments数据集的一个版本，并使用CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用 `ImageDataset` 类的 `create` 方法创建 `Dataset` 资源，该方法需要以下参数：

- `display_name`：`Dataset` 资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件列表，用于将数据项导入 `Dataset` 资源。
- `import_schema_uri`：数据项的数据标记模式：
  - `single_label`：二进制和多类分类
  - `multi_label`：多标签多类分类
  - `bounding_box`：目标检测
  - `image_segmentation`：分割

了解更多关于 [ImageDataset](https://cloud.google.com/vertex-ai/docs/datasets/prepare-image)。

In [ ]:
dataset = aiplatform.ImageDataset.create(
    display_name="flowers_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

### 创建和运行训练流水线

要训练一个AutoML模型，您需要执行两个步骤：1) 创建一个训练流水线，和 2) 运行这个流水线。

#### 创建训练流水线

使用`AutoMLImageTrainingJob`类来创建一个AutoML训练流水线，需要以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `prediction_type`：要为模型训练的任务类型。
  - `classification`：图像分类模型。
  - `object_detection`：图像目标检测模型。
- `multi_label`：如果是分类任务，是单标签（`False`）还是多标签（`True`）。
- `model_type`：部署模型的类型。
  - `CLOUD`：在Google Cloud上部署
  - `CLOUD_HIGH_ACCURACY_1`：优化精度，适用于在Google Cloud上部署
  - `CLOUD_LOW_LATENCY_`：优化延迟，适用于在Google Cloud上部署
  - `MOBILE_TF_VERSATILE_1`：在边缘设备上部署
  - `MOBILE_TF_HIGH_ACCURACY_1`：优化精度，适用于在边缘设备上部署
  - `MOBILE_TF_LOW_LATENCY_1`：优化延迟，适用于在边缘设备上部署
- `base_model`：（可选）从现有的`Model`资源进行迁移学习—仅支持图像分类。

实例化的对象是用于训练作业的DAG（有向无环图）。

In [ ]:
dag = aiplatform.AutoMLImageTrainingJob(
    display_name="flowers_" + TIMESTAMP,
    prediction_type="classification",
    multi_label=False,
    model_type="MOBILE_TF_LOW_LATENCY_1",
    base_model=None,
)

print(dag)

#### 运行训练流水线

接下来，通过调用方法 `run` 来运行创建的DAG，以启动训练作业，并使用以下参数：

- `dataset`：用于训练模型的 `Dataset` 资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留置数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `budget_milli_node_hours`：（可选）以毫节点小时（1000 = 节点小时）为单位指定的最大训练时间。
- `disable_early_stopping`：如果为 `True`，则在服务认为无法进一步提高模型目标测量之前，可能会在使用整个预算之前完成训练。

完成 `run` 方法后，将返回 `Model` 资源。

训练流水线的执行将需要超过30分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="flowers_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

## 查看模型评估分数

在模型训练完成后，您可以使用`list_model_evaluations()`方法查看其评估分数。该方法将返回每个评估切片的迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

部署模型

接下来，部署您的模型用于在线预测。要部署模型，请调用 `deploy` 方法。

In [ ]:
endpoint = model.deploy()

发送一个在线预测请求

向您部署的模型发送一个在线预测。

获取测试项目

您将使用数据集中的一个任意示例作为测试项目。不必担心该示例很可能在训练模型时已经被使用过。您只需看看如何进行预测。

In [ ]:
test_item = !gsutil cat $IMPORT_FILE | head -n1
if len(str(test_item[0]).split(",")) == 3:
    _, test_item, test_label = str(test_item[0]).split(",")
else:
    test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

### 进行预测

现在您的`Model`资源已部署到`Endpoint`资源上，您可以通过向Endpoint资源发送预测请求来进行在线预测。

#### 请求

在这个例子中，由于您的测试项目位于公共Cloud Storage存储桶中，您需要将其复制到您的存储桶中，并使用`Cloud Storage SDK`读取图像的内容。为了将测试数据传递给预测服务，您需要将字节编码为base64，这样可以使内容在网络上传输二进制数据时不易被修改。

每个实例的格式为：

    { 'content': { 'b64': base64编码的字节 } }

由于`predict()`方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目的列表发送。

#### 响应

`predict()`调用的响应是一个Python字典，包含以下条目：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `displayNames`：每个类标签的类名。
- `confidences`：每个类标签的预测置信度，介于0和1之间。
- `deployed_model_id`：执行预测的部署的Model资源的Vertex AI标识符。

In [ ]:
import base64

from google.cloud import storage

# Copy the test image to the Cloud storage bucket as "test.jpg"
test_image_local = "{}/test.jpg".format(BUCKET_URI)
! gsutil cp $test_item $test_image_local

# Download the test image in bytes format
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(bucket_name=BUCKET_NAME)
test_content = bucket.get_blob("test.jpg").download_as_bytes()

# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{"content": base64.b64encode(test_content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

print(prediction)

#### 使用[GFile](https://www.tensorflow.org/api_docs/python/tf/io/gfile/GFile)的替代方法

另外，可以使用tensorflow-io库中的[GFile](https://www.tensorflow.org/api_docs/python/tf/io/gfile/GFile)方法直接从云存储中读取数据。以下代码段执行相同的操作：

```
import base64
import tensorflow as tf

# 使用GFile读取测试文件
with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

# 每个实例的格式应符合部署模型的预测输入模式
instances = [{"content": base64.b64encode(content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

print(prediction)
```
然而，`tf.io.gfile.GFile`支持多种文件系统实现，包括本地文件、Google Cloud存储（使用gs://前缀）和HDFS（使用hdfs://前缀）。

取消部署模型

当您完成预测时，需要从`端点`资源中取消部署模型。这将取消所有计算资源，并停止为已部署的模型结算费用。

In [ ]:
endpoint.undeploy_all()

## 导出为Edge模型

您可以将AutoML云模型导出为`Edge`模型，然后将其自定义部署到边缘设备或本地下载。使用`export_model()`方法将模型导出到Cloud Storage，需要传入以下参数：

- `artifact_destination`：用于存储SavedFormat模型文件的Cloud Storage位置。
- `export_format_id`：要保存模型格式的格式。对于AutoML云，只有一个选项：
   - `tf-saved-model`：用于部署到容器的TensorFlow SavedFormat。
   - `tflite`：用于部署到边缘或移动设备的TensorFlow Lite。
   - `edgetpu-tflite`：用于TPU的TensorFlow Lite。
   - `tf-js`：用于网络客户端的TensorFlow。
   - `coral-ml`：用于Coral设备。

- `sync`：是否以同步或异步方式执行操作。

In [ ]:
response = model.export_model(
    artifact_destination=BUCKET_URI, export_format_id="tflite", sync=True
)

model_package = response["artifactOutputUri"]

删除模型

方法'delete()'将删除模型。

In [ ]:
model.delete()

删除数据集

方法'delete()'会删除数据集。

In [ ]:
dataset.delete()

删除该端点

方法 'delete()' 将删除该端点。

In [ ]:
endpoint.delete()

自动机器学习表格模型

自动机器学习可以训练以下类型的表格模型：

- 分类
- 回归
- 预测

一个模型可以被训练以便自动部署到云端，或者导出以便手动部署到云端。

了解更多关于 [AutoML 模型类型](https://cloud.google.com/vertex-ai/docs/start/automl-model-types)

### 数据准备

对于表格数据，Vertex AI的`Dataset`资源有一些要求。

- 必须在CSV文件或BigQuery表中。

了解更多关于[准备表格数据](https://cloud.google.com/vertex-ai/docs/datasets/prepare-tabular)。

#### CSV

对于表格模型，CSV文件有一些要求：

- 第一行必须是标题 -- 请注意，这与图片、文本和视频不同，这些类型的要求是没有标题。
- 除了一个列是特征之外，所有其他列都是特征。
- 一个列是标签，您在随后创建训练流水线时将指定。

##### ML_USE

每行可能还可以指定在数据集进行训练拆分时将数据分配给哪个拆分；否则，数据集将被随机分割为80/10/10。

`ml_use`指定是通过在列前添加一个用于指定分配的列来指定的 -- 作为第一列。值可以是以下之一：training, test, or validation。

请将BigQuery培训数据的位置设为`IMPORT_FILE`变量的位置。

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"
BQ_TABLE = "bigquery-public-data.samples.gsod"

### 创建数据集

#### BigQuery 输入数据

接下来，使用 `TabularDataset` 类的 `create` 方法为 `Dataset` 资源创建数据集，其中包括以下参数：

- `display_name`：`Dataset` 资源的可读名称。
- `bq_source`：将数据项从 BigQuery 表导入到 `Dataset` 资源中。
- `labels`：用户定义的元数据。在此示例中，您会存储包含用户定义数据的 Cloud Storage 存储桶的位置。

了解有关[从 BigQuery 表中创建 TabularDataset](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_bigquery_sample-python)的更多信息。

In [ ]:
dataset = aiplatform.TabularDataset.create(
    display_name="gsod_" + TIMESTAMP,
    bq_source=[IMPORT_FILE],
    labels={"user_metadata": BUCKET_NAME},
)

label_column = "mean_temp"

print(dataset.resource_name)

In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "year"}},
    {"auto": {"column_name": "month"}},
    {"auto": {"column_name": "day"}},
]

label_column = "mean_temp"

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1) 创建一个训练管道，2) 运行该管道。

#### 创建训练管道

使用`AutoMLTabularTrainingJob`类创建一个AutoML训练管道，使用以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `optimization_prediction_type`：为模型训练指定的任务类型。
  - `classification`：一个表格分类模型。
  - `regression`：一个表格回归模型。
- `column_transformations`：（可选）应用于输入列的转换。
- `optimization_objective`：最小化或最大化的优化目标。
  - 二分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类别分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

In [ ]:
dag = aiplatform.AutoMLTabularTrainingJob(
    display_name="gsod_" + TIMESTAMP,
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_transformations=TRANSFORMATIONS,
)

print(dag)

运行训练管道

接下来，您可以通过调用`run`方法来运行创建的DAG，以启动训练作业，以下是参数：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `target_column`：要作为标签进行训练的列名。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果为`True`，则在服务认为无法进一步改进模型目标测量之前，训练可能会在使用整个预算之前完成。

`run`方法完成后会返回`Model`资源。

执行训练管道将需要超过30分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="gsod_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column="mean_temp",
)

## 回顾模型评估分数

在模型训练完成后，您可以使用`list_model_evaluations()`方法回顾其评估分数。该方法将为每个评估切片返回一个迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

部署模型

接下来，为在线预测部署您的模型。要部署模型，您需要调用 `deploy` 方法，并提供以下参数：

- `machine_type`：计算机类型。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

### 下线模型

当您完成预测时，您需要将模型从“端点”资源中下线。这将取消所有计算资源并终止部署模型的计费。

In [ ]:
endpoint.undeploy_all()

将云模型导出

您可以将AutoML云模型导出为TensorFlow SavedFormat模型，然后可以自定义部署到云存储或本地下载。使用`export_model()`方法将模型导出到云存储，该方法接受以下参数：

- `artifact_destination`：保存SavedFormat模型工件的Cloud Storage位置。
- `export_format_id`：保存模型格式的格式。对于AutoML云，只有一个选项：
   - `tf-saved-model`：TensorFlow SavedFormat
- `sync`：是否同步或异步执行操作。

In [ ]:
response = model.export_model(
    artifact_destination=BUCKET_URI, export_format_id="tf-saved-model", sync=True
)

model_package = response["artifactOutputUri"]

删除模型

方法 'delete()' 将删除模型。

In [ ]:
model.delete()

删除数据集

方法'delete（）'将删除数据集。

In [ ]:
dataset.delete()

删除端点

方法 'delete()' 将删除端点。

In [ ]:
endpoint.delete()

AutoML可以训练以下类型的文本模型：

- 分类
- 情感分析
- 实体提取

了解更多关于[AutoML模型类型](https://cloud.google.com/vertex-ai/docs/start/automl-model-types)

数据准备

对于文本，Vertex AI的“数据集”资源有一些要求。

- 文本示例必须存储在CSV或JSONL文件中。

了解更多关于[准备文本数据](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text)。

#### CSV

对于文本分类，CSV文件有一些要求：

- 没有标题。
- 第一列是文本示例或云存储中文本文件的路径（.txt后缀）。
- 第二列是标签。
- 任何剩余的列都是用于多标签文本分类的附加标签。

对于文本情感分析，CSV文件有一些要求：

- 没有标题。
- 第一列是文本示例或云存储中文本文件的路径（.txt后缀）。
- 第二列是情感值。
- 第三列是最大可能的情感值。

##### ML_USE

当数据集进行训练集分割时，每一行可能会指定将数据项目分配给哪个分割；否则，数据集将被随机分割：80/10/10。

`ml_use`分配是通过在第一列之前添加一个列来指定的。值可以是以下之一：training，test或validation。

JSONL 

对于文本分类，JSONL文件有几个要求：

- 每个数据项是一个独立的JSON对象，位于单独的一行。
- 键/值对`text_gcs_uri`是文本文件在Cloud Storage中的路径。
- 键/值对`text_content`是指定文本的另一种方式，以内联方式表示。
- 键/值对`display_name`是文本的标签。

{
  "classification_annotation": {
    "display_name": label
  },
  "text_content": text
}
{
  "classification_annotation": {
    "display_name": label
  },
  "text_gcs_uri": "gcs_uri_to_file"
}

对于多标签，标签被指定为`display_name`键/值对的列表：

      'classification_annotations': [
          { 'display_name': label1
          },
          { 'display_name': labelN
          },
       ]

对于文本情感分析，JSONL文件有几个要求：

- 每个数据项是一个独立的JSON对象，位于单独的一行。
- 键/值对`text_gcs_uri`是文本文件在Cloud Storage中的路径。
- 键/值对`text_content`是指定文本的另一种方式，以内联方式表示。
- 键/值对`sentiment`是情感值，为大于0的整数值。
- 键/值对`sentiment_max`是情感的最大可能值。

{
  "sentiment_annotation": {
    "sentiment": number,
    "sentiment_max": number
  },
  "text_content": text,
}
{
  "sentiment_annotation": {
    "sentiment": number,
    "sentiment_max": number
  },
  "text_gcs_uri": "gcs_uri_to_file"
}


对于文本实体提取，JSONL文件还有一些要求：

- 每个数据项是一个独立的JSON对象，位于单独的一行。
- 键/值对`text_gcs_uri`是文本文件在Cloud Storage中的路径。
- 键/值对`text_content`是指定文本的另一种方式，以内联方式表示。
- 键/值对`start_offset`是文本起始字符的偏移量。
- 键/值对`end_offset`是文本结束字符的偏移量。
- 键/值对`display_name`是文本的标签。

{
    "text_segment_annotations": [
      {
        "start_offset":number,
        "end_offset":number,
        "display_name": label
      },
      ...
    ],
    "textContent": "inline_text"
}
{
    "textSegmentAnnotations": [
      {
        "start_offset": number,
        "end_offset": number,
        "displayName": label
      },
      ...
    ],
    "text_gcs_uri": "gcs_uri_to_file"
}

ML_USE

每个JSONL对象还可以指定将数据项分配给哪个拆分用于训练；否则，数据集将被随机分割：80/10/10。

"data_item_resource_labels": {
      "aiplatform.googleapis.com/ml_use": "training|test|validation"
    }

*注意*: 字典键字段也可以选择使用驼峰命名法，例如，'text_gcs_uri'也可以是'textGcsUri'。

云存储训练数据的位置。

现在将变量 `IMPORT_FILE` 设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-data/NL-classification/happiness.csv"

#### 快速查看您的数据

本教程使用存储在公共云存储桶中的Happy Moments数据集版本，使用CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例数量，然后查看前几行数据。

In [ ]:
FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

###创建数据集

接下来，使用`TextDataset`类的`create`方法为`Dataset`资源创建数据集，该方法接受以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标记模式。
  - `single_label`：二元和多类别分类
  - `multi_label`：多标签多类别分类
  - `sentiment`：情感分析
  - `extraction`：实体提取

了解更多关于[TextDataset](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text)。

In [ ]:
dataset = aiplatform.TextDataset.create(
    display_name="happydb_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
)

print(dataset.resource_name)

### 创建和运行训练流水线

要训练一个AutoML模型，您需要执行两个步骤：1) 创建一个训练流水线，2) 运行这个流水线。

#### 创建训练流水线

通过`AutoMLTextTrainingJob`类创建一个AutoML训练流水线，需要提供以下参数：

- `display_name`: 用于`TrainingJob`资源的易读名称。
- `prediction_type`: 用于训练模型的任务类型。
  - `classification`: 文本分类模型。
  - `sentiment`: 文本情感分析模型。
  - `extraction`: 文本实体提取模型。
- `multi_label`: 如果是分类任务，是单标签(False)还是多标签(True)。
- `sentiment_max`: 如果是情感分析任务，则是情感值的最大值。

In [ ]:
dag = aiplatform.AutoMLTextTrainingJob(
    display_name="happydb_" + TIMESTAMP,
    prediction_type="classification",
    multi_label=False,
)

print(dag)

#### 运行训练流水线

接下来，您可以通过调用 `run` 方法并传入以下参数来运行创建的DAG以启动训练作业：

- `dataset`：用于训练模型的 `Dataset` 资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集所占比例。
- `test_fraction_split`：用于测试（留置数据）的数据集所占比例。
- `validation_fraction_split`：用于验证的数据集所占比例。

完成后，`run` 方法将返回 `Model` 资源。

训练流水线的执行将需要大约> 30分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="happydb_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

##回顾模型评估分数

在模型训练完成后，您可以使用 `list_model_evaluations()` 方法回顾它的评估分数。该方法将返回每个评估切片的迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

部署模型

接下来，部署您的模型以进行在线预测。要部署模型，您需要调用`deploy`方法。

In [ ]:
endpoint = model.deploy()

取消部署模型

当您完成预测后，您可以从`Endpoint`资源中取消部署模型。这将取消所有计算资源并停止对部署模型的计费。

In [ ]:
endpoint.undeploy_all()

### 删除模型

方法 'delete()' 将删除模型。

In [ ]:
model.delete()

删除数据集

方法'delete（）'将删除数据集。

In [ ]:
dataset.delete()

请删除终端点

方法 'delete()' 将删除终端点。

In [ ]:
endpoint.delete()

自动机器学习视频模型

AutoML可以训练以下类型的视频模型：

- 分类
- 物体跟踪
- 动作识别

模型可以被训练用于部署到云端，也可以被导出到边缘设备。

了解更多关于[AutoML模型类型](https://cloud.google.com/vertex-ai/docs/start/automl-model-types)。

数据准备

Vertex AI 用于文本的“数据集”资源对您的文本数据有几个要求。

- 文本示例必须存储在 CSV 或 JSONL 文件中。

详细了解如何[准备视频数据](https://cloud.google.com/vertex-ai/docs/datasets/prepare-video)。

#### CSV

对于视频分类，CSV文件有一些要求：

- 没有标题。
- 第一列是视频文件的 Cloud Storage 路径。
- 第二列是标签。
- 第三列是视频中要分类的开始时间（秒）。
- 第四列是视频中要分类的结束时间（秒）。

对于多标签分类，每个标签是单独的行条目。

对于视频对象跟踪，CSV文件有一些要求：

- 没有标题。
- 第一列是视频文件的 Cloud Storage 路径。
- 第二列是标签。
- 第三列是未使用的（空白）。
- 第四列是视频中开始跟踪对象的时间（秒）。
- 第五到八列是要跟踪的对象的顶点。
    - x_min
    - y_min
    - x_max
    - y_max

对于动作识别，CSV文件有一些要求：

- 没有标题。
- 每行可以是以下四种格式之一：

VIDEO_URI, TIME_SEGMENT_START, TIME_SEGMENT_END, LABEL, ANNOTATION_FRAME_TIMESTAMP

VIDEO_URI, , , LABEL, ANNOTATION_FRAME_TIMESTAMP

VIDEO_URI, TIME_SEGMENT_START, TIME_SEGMENT_END, LABEL, ANNOTATION_SEGMENT_START, ANNOTATION_SEGMENT_END

VIDEO_URI, , , LABEL, ANNOTATION_SEGMENT_START, ANNOTATION_SEGMENT_END


##### ML_USE

每一行还可以指定在数据集用于训练时要分配给数据项的分割方式；否则，数据集将被随机分割为 80/10/10。

`ml_use` 的分配通过在指定分配的列之前添加一列来指定 -- 作为第一列。该值可以是以下之一：training，或 test。

JSONL

对于视频分类，CSV文件有一些要求：

- 每个数据项是一个单独的JSON对象，放在单独的一行上。
- 键/值对`video_gcs_uri`是文本文件的云存储路径。
- 键/值对`display_name`是文本的标签。
- 键/值对`start_time`是分类的开始时间（秒）。
- 键/值对`end_time`是分类的结束时间（秒）。

{
    "video_gcs_uri": video,
    "time_segment_annotations": [{
        "display_name": label,
        "start_time": "start_time_of_segment",
        "end_time": "end_time_of_segment"
    }]
}

对于视频对象跟踪，CSV文件有一些要求：

- 每个数据项是一个单独的JSON对象，放在单独的一行上。
- 键/值对`video_gcs_uri`是文本文件的云存储路径。

{
    "video_gcs_uri": video,
    "temporal_bounding_box_annotations": [{
        "display_name": label,
        "x_min": "bounding框的最左坐标",
        "x_max": "bounding框的最右坐标",
        "y_min": "bounding框的最上坐标",
        "y_max": "bounding框的最下坐标",
        "time_offset": "检测到对象的时间帧"
    }]
}

对于视频动作识别，CSV文件有一些要求：

- 每个数据项是一个单独的JSON对象，放在单独的一行上。
- 键/值对`video_gcs_uri'是文本文件的云存储路径。

{
  "video_gcs_uri': video,
  "time_segments": [{
    "start_time": "完全注释段的开始时间",
    "end_time": "段的结束时间"}],
  "time_segment_annotations": [{
    "display_name": label,
    "start_time": "段的开始时间",
    "end_time": "段的结束时间"
  }]
}

ML_USE

每个JSONL对象还可以指定将数据项分配到哪个拆分中进行训练；否则，数据集将被随机分割：80/20。

"data_item_resource_labels": {
  "aiplatform.googleapis.com/ml_use": "training|test"
}

*注意*：字典键字段也可以使用驼峰命名法。例如，'video_gcs_uri'也可以是'videoGcsUri'。

云存储培训数据的位置。

现在将变量`IMPORT_FILE`设置为Cloud Storage中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://automl-video-demo-data/hmdb_split1_5classes_train_inf.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的Happy Moments数据集的一个版本，使用CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例数量，然后查看前几行。

In [ ]:
FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`VideoDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标记模式。
  - `classification`：二元和多类分类
  - `object_tracking`：对象追踪
  - `action_recognition`：动作识别

了解更多关于[VideoDataset](https://cloud.google.com/vertex-ai/docs/datasets/prepare-video)。

In [ ]:
dataset = aiplatform.VideoDataset.create(
    display_name="human_motion_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.video.classification,
)

print(dataset.resource_name)

### 创建并运行训练流程

为训练 AutoML 模型，您需要执行两个步骤：1) 创建一个训练流程，2) 运行这个流程。

#### 创建训练流程

使用 `AutoMLVideoTrainingJob` 类可以创建一个 AutoML 训练流程，具有以下参数：

- `display_name`：`TrainingJob` 资源的人类可读名称。
- `prediction_type`：为模型训练指定的任务类型。
  - `classification`：视频分类模型。
  - `object_tracking`：视频目标追踪模型。
  - `action_recognition`：视频动作识别模型。

In [ ]:
dag = aiplatform.AutoMLVideoTrainingJob(
    display_name="human_motion_" + TIMESTAMP,
    prediction_type="classification",
)

print(dag)

#### 运行训练管道

接下来，通过调用`run`方法并传入以下参数来运行创建的DAG以开始训练作业：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留置数据）的数据集百分比。

完成`run`方法后，将返回`Model`资源。

训练管道的执行将需要超过30分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="human_motion_" + TIMESTAMP,
    training_fraction_split=0.8,
    test_fraction_split=0.2,
)

## 查看模型评估分数

在模型训练完成后，您可以使用`list_model_evaluations()`方法查看其评估分数。该方法将为每个评估切片返回一个迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

删除模型

方法'delete（）'将删除模型。

In [ ]:
model.delete()

删除数据集

方法'delete()'将删除数据集。

In [ ]:
dataset.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以 [删除您用于本教程的Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI